# Monte Carlo Convolution Exercise

In this exercise you will implement a simple version of the Monte Carlo Convolution layers, create a network with them, and train it to segment point clouds from the **ShapeNet Part** dataset (https://shapenet.cs.stanford.edu/iccv17/).

First load the necessary dependencies by executing the following code:

In [0]:
#Numpy
import numpy as np
#Library used to load the data
import h5py

#!pip install -q tensorflow-gpu==2.0.0-beta1
try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

# Data preparation

The first thing we will need to do is downloading the data. 

**ShapeNet** is a dataset of 12k CAD objects from 16 different classes. Each class has several parts that can be segmented, being 50 different parts in total.

We have prepared a sampled version of ShapeNet Part in which each object is sampled with 512 points. In order to download these files execute the following commands:

In [0]:
!wget https://www.dropbox.com/s/taos2s389ikli6d/shapenet.zip
!unzip shapenet.zip

If everything went well you should see the hdf5 file in the associated files of the notebook.

Here you can see an image with some segmented point clouds from the data set, for different sampling methods.

![texto alternativo](https://www.uni-ulm.de/fileadmin/website_uni_ulm/iui.inst.100/institut/Papers/viscom/2018/hermosilla2018mccnn/hermosilla2018-mccnn-segres.jpg)

Now, we will prepare the data for training. First we will load the hdf5 binary file and extract the different datasets.

For each point in each model we have the 3D position, the class label, the list of neighboring points and the category of object the point belongs to.

In [0]:
dataset = h5py.File("shapenet.hdf5")

#Training data.
x_train = dataset['train_data'][:] # 3D point coordinates.
y_train = dataset['train_labels'][:] # Point label (0-50).
pdf_train = dataset['train_pdfs'][:] # PDF value of the point.
n_train = dataset['train_neighs'][:] # Indices of the neighboring points.
cat_train = dataset['train_cats'][:] # Category of the object.

#Validation data.
x_val = dataset['val_data'][:] # 3D point coordinates.
y_val = dataset['val_labels'][:] # Point label (0-50).
pdf_val = dataset['val_pdfs'][:] # PDF value of the point.
n_val = dataset['val_neighs'][:] # Indices of the neighboring points.
cat_val = dataset['val_cats'][:] # Category of the object.

#Test data.
x_test = dataset['test_data'][:] # 3D point coordinates.
y_test = dataset['test_labels'][:] # Point label (0-50).
pdf_test = dataset['test_pdfs'][:] # PDF value of the point.
n_test = dataset['test_neighs'][:] # Indices of the neighboring points.
cat_test = dataset['test_cats'][:] # Category of the object.

print("Point cloud training:", x_train.shape[0])
print("Point cloud validation:", x_val.shape[0])
print("Point cloud testing:", x_test.shape[0])

# Spatial convolution

In the next code we will define a Monte Carlo Convolution layer. These layers compute the following equation for all points in the point cloud and for each output feature:

$(f\star k)_o(x)=\frac{1}{N_i}\sum_{i}^{N_i}\sum_{j}^{F_{in}}\frac{f_{ij}k_{jo}(x_i-x)}{p(x_i)}$

where $f_{ij}$ is the input feature $j$ associated with the neighboring point $x_i$, $k_{jo}(x_i-x)$ is the value of the kernel for the feature $j$ and the position $x_i$ with respect to the center point $x$, $p(x_i|x)$ is the probability density function associated with point $x_i$, $N_i$ is the number of neighboring points to $x$, and $F_{in}$ is the number of input features.

We will represent each kernel $k_{jo}$ with an MLP. However, with this design we have to create a different kernel for each input feature and output feature. This could not be memory efficient. Instead, we are going to share the first part of the MLP (first hidden layer) among all the kernels in the layer.

$MLP_{jo}(x) = \sum_{w}^{A}H1_{w}(x)W_{wjo}$

Here, the $MLP_{jo}$ represents the convolution kernel for the input feature $j$ and output feature $o$. $H1_w$ is the hidden neuron $w$ and $W_{wjo}$ are the weight applied to compute the final output. You can see that, since the first layer is shared, $H1_w$ does not depend on the input or output feature, it is the same for all the kernels. If we plugin this definition in our previous definition of convolution we get:

$(f\star k)_{o}(x)=\frac{1}{N_i}\sum_{i}^{N_i}\sum_{j}^{F_{in}}\frac{f_{ij}MLP_{jo}(x_i-x)}{p(x_i)}=$

$=\frac{1}{N_i}\sum_{i}^{N_i}\sum_{j}^{F_{in}}\sum_{w}^{A}\frac{f_{ij}H1_{w}(x_i-x)W_{wjo}}{p(x_i)}=$

$=\sum_{j}^{F_{in}}\sum_{w}^{A} \left( \sum_{i}^{N_i}\frac{f_{ij}H1_{w}(x_i-x)}{p(x_i)N_i}\right) W_{wjo}$

$a_{jw}=\sum_{i}^{N_i}\frac{f_{ij}H1_{w}(x_i-x)}{p(x_i)N_i}$

$(f\star k)_{o}(x)=\sum_{j}^{F_{in}}\sum_{w}^{A} a_{jw} W_{wjo}$

Now we can compute $a_{jw}$ independently of the output feature and then use a simple matrix multiplication to compute the final convoluted output features.

In this part of the exercise you will have to implement a spatial convolution operation using the low level API of tensorflow 2.0. You will need to look at functions such as: tf.gather_nd, tf.reshape, tf.reduce_sum, ...


In [0]:
def mc_conv(p_in_pts, 
            p_in_pdf, 
            p_in_neighbors, 
            p_in_features, 
            p_num_in_features, 
            p_num_out_features):
  """ Method to create a monte carlo convolution layer.
  
  Params:
    p_in_pts (BxNx3): Tensor with the points for each model in the batch.
    p_in_pdf (BxN): Tensor with the pdf values for each point.
    p_in_neighbors (BxNx16): Tensor with the indices of the closest neighbors
      for each point.
    p_in_features (BxNxF): Tensor with the input features.
    p_num_out_features (int): Number of output features.
  Return:
    Tensor with the new convolved features.
  """
  # Num Axes
  num_axes = 32

  # Prepare the indices of the neighboring points.
  batch_indices = tf.tile(tf.reshape(tf.range(tf.shape(p_in_pts)[0]), 
        (-1, 1, 1, 1)), (1, 512, 16, 1))
  expanded_neighbors = tf.expand_dims(p_in_neighbors, 3)
  idx = tf.concat([batch_indices, expanded_neighbors], axis = 3)
  
  ################# TODO
  # Get the 3D coordinates of the neighboring points and compute the difference
  # with the central point.
  neigh_pt_diffs = ...
  
  # Get the features of the neighboring points.
  neigh_features = ...
  
  # Get the pdf values of the neighboring points.
  neigh_pdfs = ...
  
  # Compute the projection of each neighbor into the axes (H1).
  h1 = ...
  
  # Compute the weighted input features for each axis (a_jw).
  a_jw = ...
  
  # Compute the final output features.
  out_features = ...
  ################# END TODO
  
  return out_features

At this point we should have the function to create a Monte Carlo convolution implemented. Now we can use this building block to create our architecture. We are going to implemented a simple architecture with 3 MC convolutions, each of them followed by a batch normalization and activation function. The first one will output 64 features, the second one 128, and the third one 256. Then, we will apply an MLP to the convoluted features obtained for each point to classify the point into the different classes. We also concatenate to the features a one hot vector with the class of the object to simplify the task.

In [0]:
#Define the inputs to keras
inputs = tf.keras.Input(shape=(512, 3), name='batch_point_cloud')
pdfs = tf.keras.Input(shape=(512), name='batch_pdfs')
neighs = tf.keras.Input(shape=(512, 16), name='batch_neighs', dtype=tf.int32)
cats = tf.keras.Input(shape=(512), name='batch_cats', dtype=tf.int32)
cats_one_hot = tf.one_hot(cats, 16)
initFeatures = tf.reshape(tf.ones_like(pdfs, dtype=tf.float32), [-1, 512, 1])

################# TODO
# Create the first layer with 64 features.
x = ...

# Create the second layer with 128 features.
x = ...

# Create the first layer with 256 features.
x = ...
################# END TODO

#Concatenate the one hot vector to the features.
x = tf.concat([x, cats_one_hot], axis=-1)

#Define the MLP to classify the points.
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

#Create the final linear + softmax layer.
outputs = tf.keras.layers.Dense(50, activation='softmax')(x)

In [0]:
#Create the model.
model = tf.keras.Model(inputs=[inputs, pdfs, neighs, cats], outputs=outputs, 
                       name='shapenet_model')

#Compile the model.
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(
                  learning_rate=0.0025, 
                  momentum=0.98),
              metrics=['accuracy'])

#Fit the model to the data.
model.fit([x_train, pdf_train, n_train, cat_train], y_train,
          batch_size=32,
          epochs=20,
          validation_data=([x_val, pdf_val, n_val, cat_val], y_val))

#Evaluate the model on the test data.
model.evaluate([x_test, pdf_test, n_test, cat_test], y_test, verbose=0)

# Resnet architecture


Now you can try variations on the architecture and learning rate to try to climb in the leaderboard. In this part of the exercise we are going to create the code to use resnet bottleneck blocks.

This blocks are composed of a 1x1 convolution that reduces the number of features to a fourth of the input, then applies a spatial convolution, and finally another 1x1 convolution is applied to the features to increase the resolution to the same shape as the input. Then, the input and the output are added together to generate the final output of the layer.

In [0]:
def resnet_bottleneck_mc_conv(
            p_in_pts, 
            p_in_pdf, 
            p_in_neighbors, 
            p_in_features, 
            p_num_in_features):
  """ Method to create a resnet bottleneck block.
  
  Params:
    p_in_pts (BxNx3): Tensor with the points for each model in the batch.
    p_in_pdf (BxN): Tensor with the pdf values for each point.
    p_in_neighbors (BxNx16): Tensor with the indices of the closest neighbors
      for each point.
    p_in_features (BxNxF): Tensor with the input features.
    p_num_in_features (int): Number of input features.
  Return:
    Tensor with the new convolved features.
  """
  # Batch norm the input
  batch_norm_in = tf.keras.layers.BatchNormalization()(p_in_features)

  ################# TODO
  # Create the first 1x1 convolution to reduce the input size 
  # to p_num_in_features//4
  x = ...
  
  # Create the spatial convolution
  x = ...

  # Create the last 1x1 convolution to increase the input size 
  # to p_num_in_features
  x = ...
  ################# END TODO

  x = tf.keras.layers.BatchNormalization()(x)

  # Return the addition of the input and output.
  return batch_norm_in + x

Now we are going to use this blocks to create a new network architecture. First, we will create a monte carlo convolution to create the initial features (64), then we will apply a resnet block. After, we will apply a 1x1 convolution to increase the number of features to 128 and then another resnet block. Lastly, we will use a 1x1 convolution to increase the resolution to 256 features and we will apply the last resnet block.


In [0]:
#Define the inputs to keras
inputs = tf.keras.Input(shape=(512, 3), name='batch_point_cloud')
pdfs = tf.keras.Input(shape=(512), name='batch_pdfs')
neighs = tf.keras.Input(shape=(512, 16), name='batch_neighs', dtype=tf.int32)
cats = tf.keras.Input(shape=(512), name='batch_cats', dtype=tf.int32)
cats_one_hot = tf.one_hot(cats, 16)
initFeatures = tf.reshape(tf.ones_like(pdfs, dtype=tf.float32), [-1, 512, 1])

################# TODO
# Create the first spatial convolution with 64 output features.
x = ...

# Create the first resnet block.
x = ...

# Create the 1x1 convolution to increase the number of features to 128.
x = ...

# Create the second resnet block.
x = ...

# Create the 1x1 convolution to increase the number of features to 256.
x = ...

# Create the third resnet block.
x = ...

################# END TODO

#Concatenate the one hot vector to the features.
x = tf.concat([x, cats_one_hot], axis=-1)

#Define the MLP to classify the points.
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

#Create the final linear + softmax layer.
outputs = tf.keras.layers.Dense(50, activation='softmax')(x)

In [0]:
#Create the model.
model = tf.keras.Model(inputs=[inputs, pdfs, neighs, cats], outputs=outputs, 
                       name='shapenet_model')

#Compile the model.
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(
                  learning_rate=0.0025, 
                  momentum=0.98),
              metrics=['accuracy'])

#Fit the model to the data.
model.fit([x_train, pdf_train, n_train, cat_train], y_train,
          batch_size=32,
          epochs=20,
          validation_data=([x_val, pdf_val, n_val, cat_val], y_val))

#Evaluate the model on the test data.
model.evaluate([x_test, pdf_test, n_test, cat_test], y_test, verbose=0)